In [1]:
import os 
os.chdir(os.path.dirname(os.path.abspath('./')))
print(os.getcwd())

/Users/jberg/repositories/Metaboverse/cli/metaboverse_cli


In [2]:
import analyze.prepare_data as prepare_data
import analyze.__main__ as analyze
import analyze.model as model

In [3]:
import mapper.__main__ as mapper

In [4]:
import utils as utils

In [ ]:
output_file = mapper.download_hmbd_reference(
    output_dir='.')

In [5]:
output_file = 'hmdb_metabolites.xml'

In [6]:
hmdb_dictionary, display_dictionary, mapping_dictionary = mapper.parse_hmdb_synonyms(
        output_file=output_file
    )

Parsing HMDB metabolite records...


In [8]:
mapping_dictionary

{'(2s)-2-amino-3-(1-methyl-1h-imidazol-4-yl)propanoate': '1methylhistidine',
 '(2s)-2-amino-3-(1-methyl-1h-imidazol-4-yl)propanoic acid': '1methylhistidine',
 '1 methylhistidine': '1methylhistidine',
 '1-methyl histidine': '1methylhistidine',
 '1-methyl-histidine': '1methylhistidine',
 '1-methyl-l-histidine': '1methylhistidine',
 '1-methylhistidine': '1methylhistidine',
 '1-methylhistidine dihydrochloride': '1methylhistidine',
 '1-mhis': '1methylhistidine',
 '1-n-methyl-l-histidine': '1methylhistidine',
 '1methylhistidine': '1methylhistidine',
 '1methylhistidinedihydrochloride': '1methylhistidine',
 '1methyllhistidine': '1methylhistidine',
 '1mhis': '1methylhistidine',
 '1nmethyllhistidine': '1methylhistidine',
 '2s2amino31methyl1himidazol4ylpropanoate': '1methylhistidine',
 '2s2amino31methyl1himidazol4ylpropanoicacid': '1methylhistidine',
 'HMDB0000001': '1methylhistidine',
 'hmdb0000001': '1methylhistidine',
 'l-1-methylhistidine': '1methylhistidine',
 'l1methylhistidine': '1methylhi

In [9]:
mapping_db = {
        'hmdb_dictionary': hmdb_dictionary,
        'display_dictionary': display_dictionary,
        'mapping_dictionary': mapping_dictionary
    }

print('Writing database to file...')
utils.write_database(
    output='.',
    file='metabolite_mapping.pickle',
    database=mapping_db)

Writing database to file...


In [ ]:
from __future__ import print_function
import pandas as pd
import xml.etree.ElementTree as et
import requests
import zipfile
import pickle
import io
import os

xml_tag='{http://www.hmdb.ca}'

In [ ]:
print("Parsing HMDB metabolite records...")
#os.remove(output_file)

hmdb_dictionary = {}
display_dictionary = {}
mapping_dictionary = {}

In [ ]:
print("Parsing HMDB metabolite records...")
hmdb_contents = et.parse(output_file)
contents = hmdb_contents.getroot()

In [ ]:
hmdb_dictionary = {}
display_dictionary = {}
mapping_dictionary = {}

for x in contents:

    if x.tag == xml_tag + 'metabolite':

        name = ''
        synonyms = set()
        display_synonyms = set()

        for y in x:
            if y.text != None:
                simple_string = ''.join(
                    str(c).lower() for c in y.text if c.isalnum()
                )
            else:
                simple_string = 'None'

            if y.tag == xml_tag + 'name':
                name = simple_string
                synonyms.add(simple_string)
                synonyms.add(str(y.text).lower())
                display_synonyms.add(str(y.text))

            if y.tag == xml_tag + 'synonyms':
                for child in y:
                    simple_child = ''.join(
                        str(c).lower() for c in child.text if c.isalnum()
                    )
                    synonyms.add(simple_child.lower())
                    synonyms.add(str(child.text).lower())
                    display_synonyms.add(str(child.text))

            if y.tag == xml_tag + 'iupac_name':
                synonyms.add(simple_string)
                synonyms.add(str(y.text).lower())
                display_synonyms.add(str(y.text))

            if y.tag == xml_tag + 'traditional_iupac':
                synonyms.add(simple_string)
                synonyms.add(str(y.text).lower())
                display_synonyms.add(str(y.text))
                
            if y.tag == xml_tag + 'accession':
                synonyms.add(simple_string)
                synonyms.add(str(y.text).lower())
                synonyms.add(str(y.text))
                display_synonyms.add(str(y.text))

        if name != '':
            hmdb_dictionary[name] = sorted(list(synonyms))
            display_dictionary[name] = sorted(list(display_synonyms))
            for l in hmdb_dictionary[name]:
                mapping_dictionary[l] = name